###  Import the Libraries

In [44]:
import openai
import os
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text

In [45]:
openai.api_key = os.getenv('OPENAI_API_KEY')

###  Read the csv file

In [46]:
df = pd.read_csv('sales_data_sample.csv')
df

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,10350,20,100.00,15,2244.40,12/2/2004 0:00,Shipped,4,12,2004,...,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego,Small
2819,10373,29,100.00,1,3978.51,1/31/2005 0:00,Shipped,1,1,2005,...,Torikatu 38,NaN,Oulu,NaN,90110,Finland,EMEA,Koskitalo,Pirkko,Medium
2820,10386,43,100.00,4,5417.57,3/1/2005 0:00,Resolved,1,3,2005,...,"C/ Moralzarzal, 86",NaN,Madrid,NaN,28034,Spain,EMEA,Freyre,Diego,Medium
2821,10397,34,62.24,1,2116.16,3/28/2005 0:00,Shipped,1,3,2005,...,1 rue Alsace-Lorraine,NaN,Toulouse,NaN,31000,France,EMEA,Roulet,Annette,Small


### Setting up a temporary database in the RAM

In [47]:
temp_db = create_engine('sqlite:///:memory:',echo = True)

### Push the data to temp database

In [48]:
data = df.to_sql(name ='Sales',con =temp_db)

2023-03-11 16:06:24,539 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-03-11 16:06:24,543 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-11 16:06:24,547 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-03-11 16:06:24,550 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-11 16:06:24,581 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-11 16:06:24,596 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)


20

In [ ]:
### Ingesting the table to the GPT-3

In [49]:
def create_table_definitions(df):
    prompt = """### sqlite SQL table with its properties:
    #
    # Sales({})
    #
    """.format(','.join(str(x)for x in df.columns))
    
    return prompt

In [50]:
def prompt_input():
    nlp_text = input('Enter the info you want : ')
    return nlp_text

In [51]:
def combine_prompts(df,query_prompt):
    definition = create_table_definitions(df)
    query_init_prompt = f"### A query to answer : {query_prompt} \nSELECT"
    return definition+query_init_prompt

In [52]:
def api_response(nlp_text):
    response = openai.Completion.create(
                model = 'code-davinci-002',
                prompt = combine_prompts(df,nlp_text),
                temperature = 0,
                max_tokens = 150,
                top_p = 1,
                frequency_penalty = 0,
                presence_penalty = 0,
                stop = ['#',';'])
    return response

In [53]:
def handle_response(response):
    query = response['choices'][0]['text']
    if query.startswith(' '):
        query = 'SELECT'+query
    return query                                                                

In [54]:
def fnctn():
    nlp_text = prompt_input()
    combine_prompts(df,nlp_text)
    response = api_response(nlp_text)
    query = handle_response(response)
    
    with temp_db.connect() as conn: #make connection with the database and execute the query 
        result = conn.execute(text(handle_response(response)))
    return result.all()

In [55]:
fnctn()

Entere the info you want return the sum of SALES per QTR_ID
2023-03-11 16:07:25,765 INFO sqlalchemy.engine.Engine SELECT QTR_ID, SUM(SALES) FROM Sales GROUP BY QTR_ID
2023-03-11 16:07:25,768 INFO sqlalchemy.engine.Engine [generated in 0.00278s] ()


[(1, 2350817.7300000004),
 (2, 2048120.2999999986),
 (3, 1758910.8099999994),
 (4, 3874780.01)]